In [57]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
%matplotlib inline

In [58]:
os.chdir(r'C:\Users\gbsegura\Documents\AMPD-Projects\archive\book-recommender')

## DATA LOADING

In [59]:
ratings = pd.read_csv('./data/ratings.csv', sep='\";\"', engine='python')
books = pd.read_csv('./data/books.csv', sep='\";\"', engine='python')
users = pd.read_csv('./data/users.csv', sep='\";\"', engine='python')

In [60]:
ratings.head()

,"""User-ID",ISBN,"Book-Rating"""
0,"""276725",034545104X,"0"""
1,"""276726",0155061224,"5"""
2,"""276727",0446520802,"0"""
3,"""276729",052165615X,"3"""
4,"""276729",0521795028,"6"""


In [61]:
books.head()

,"""ISBN",Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,"Image-URL-L"""
0,"""0195153448",Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,"""0002005018",Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,"""0060973129",Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,"""0374157065",Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,"""0393045218",The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [62]:
users.head()

,"""User-ID",Location,"Age"""
0,"""1","nyc, new york, usa"";NULL",None
1,"""2","stockton, california, usa","18"""
2,"""3","moscow, yukon territory, russia"";NULL",None
3,"""4","porto, v.n.gaia, portugal","17"""
4,"""5","farnborough, hants, united kingdom"";NULL",None


In [63]:
print('Shape of books -- {}'.format(books.shape))
print('Shape of ratings -- {}'.format(ratings.shape))
print('Shape of users -- {}'.format(users.shape))

Shape of books -- (271379, 8)
Shape of ratings -- (1149780, 3)
Shape of users -- (278859, 3)


## DATA CLEANUP

In [64]:
books.columns

Index(['"ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication',
       'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L"'],
      dtype='object')

In [65]:
books['"ISBN'] = books['"ISBN'].str.replace('"', '')
books['Image-URL-L"'] = books['"ISBN'].str.replace('"', '')
columns = books.columns.str.replace('"', '').values.tolist()
books.columns = columns

In [66]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,0195153448
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,0002005018
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,0060973129
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,0374157065
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,0393045218


In [67]:
ratings.columns

Index(['"User-ID', 'ISBN', 'Book-Rating"'], dtype='object')

In [68]:
ratings['"User-ID'] = ratings['"User-ID'].str.replace('"', '')
ratings['Book-Rating"'] = ratings['Book-Rating"'].str.replace('"', '')
columns = ratings.columns.str.replace('"', '').values.tolist()
ratings.columns = columns

In [69]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [70]:
users.columns

Index(['"User-ID', 'Location', 'Age"'], dtype='object')

In [71]:
users['"User-ID'] = users['"User-ID'].str.replace('"', '')
users['Age"'] = users['Age"'].str.replace('"', '')
columns = users.columns.str.replace('"', '').values.tolist()
users.columns = columns

In [72]:
users['Location'] = users['Location'].str.replace('";NULL', '')

In [74]:
location = users['Location'].str.split(',', expand=True)

In [78]:
location.head()

,0,1,2,3,4,5,6,7,8
0,nyc,new york,usa,None,None,None,None,None,None
1,stockton,california,usa,None,None,None,None,None,None
2,moscow,yukon territory,russia,None,None,None,None,None,None
3,porto,v.n.gaia,portugal,None,None,None,None,None,None
4,farnborough,hants,united kingdom,None,None,None,None,None,None


In [82]:
location.isnull().sum()

0         1
1         3
2         4
3    277351
4    278768
5    278840
6    278851
7    278855
8    278857
dtype: int64

In [83]:
location[8][pd.notnull(location[8])]

235181     india
241302     india
Name: 8, dtype: object

In [84]:
location[7][pd.notnull(location[7])]

60008           india
235181      b`lore-10
241302     maharastra
273097            usa
Name: 7, dtype: object

In [85]:
location[6][pd.notnull(location[6])]

60008            india
108113          turkey
189617         nigeria
194259     south korea
215970             usa
235181         m`nagar
241302          thane 
273097              bc
Name: 6, dtype: object

In [86]:
location[5][pd.notnull(location[5])]

60008          new delhi 
76342              mexico
85972         philippines
104037           malaysia
108113    antalya/olympos
108496                usa
122334              china
122814        philippines
189617         lagosstate
194259                302
215970            and va)
227901        philippines
235181            r`nagar
241302          ambernath
243263              india
258822     united kingdom
262288              india
273097                 ak
274065        philippines
Name: 5, dtype: object

In [91]:
users = users[~pd.notnull(location[4])]

In [105]:
location[location[2]=='usa']

,0,1,2,3,4,5,6,7,8


In [107]:
location[2].value_counts()[:20]

 usa               139188
 canada             21558
 united kingdom     18304
 germany            17052
 spain              13205
 australia          11724
 italy              11246
                     4608
 france              3474
 portugal            3371
 new zealand         3094
 netherlands         3043
 switzerland         1757
 brazil              1677
 china               1464
 sweden              1452
 india               1279
 austria             1147
 malaysia            1102
 argentina           1081
Name: 2, dtype: int64